<a href="https://colab.research.google.com/github/RealPapaZeus/News-Scraper/blob/main/news_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Start

In [ ]:
import os

os.environ["AGENTQL_API_KEY"]="efeqQVTHCP2KpSsoi49ay0rp-Okv92Q-ZzZ3uxT1aZITZLDK7L7pTg"

In [ ]:
!pip install agentql playwright nest_asyncio pandas openpyxl
!pip install playwright
!playwright install
!playwright install-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 9.6 MB/s eta 0:00:00
164.9 MiB [] 0% 0.0s164.9 MiB [] 0% 45.7s164.9 MiB [] 0% 33.1s164.9 MiB [] 0% 19.4s164.9 MiB [] 0% 15.0s164.9 MiB [] 1% 7.3s164.9 MiB [] 1% 5.6s164.9 MiB [] 2% 4.6s164.9 MiB [] 3% 4.3s164.9 MiB [] 3% 4.1s164.9 MiB [] 4% 3.8s164.9 MiB [] 4% 3.7s164.9 MiB [] 5% 3.6s164.9 MiB [] 5% 3.5s164.9 MiB [] 6% 3.6s164.9 MiB [] 6% 3.7s164.9 MiB [] 7% 3.8s164.9 MiB [] 8% 3.9s164.9 MiB [] 8% 3.8s164.9 MiB [] 9% 3.8s164.9 MiB [] 9% 4.0s164.9 MiB [] 9% 3.9s164.9 MiB [] 9% 4.0s164.9 MiB [] 10% 4.1s164.9 MiB [] 10% 4.0s164.9 MiB [] 11% 4.0s164.9 MiB [] 11% 3.9s164.9 MiB [] 12% 3.8s164.9 MiB [] 12% 3.7s164.9 MiB [] 13% 3.7s164.9 MiB [] 14% 3.6s164.9 MiB [] 15% 3.5s164.9 MiB [] 16% 3.4s164.9 MiB [] 17% 3.3s164.9 MiB [] 18% 3.2s164.9 MiB [] 19% 3.1s164.9 MiB [] 20% 3.0s164.9 MiB [] 20% 2.9s164.9 MiB [] 21% 2.9s164.9 MiB [] 21% 2.8s164.9 MiB [] 22% 2.8s16

# Load Data

In [ ]:
import pandas as pd
import agentql
from playwright.async_api import async_playwright
import asyncio
import json

# Function to load the Excel file
def load_companies_data(excel_file_path):
    df = pd.read_excel(excel_file_path)
    return df

# Scrape News Room

In [ ]:
# Function to scrape news articles for each company
async def scrape_news_articles_for_company(company, url):
    async with async_playwright() as playwright:
        browser = await playwright.chromium.launch()
        page = await agentql.wrap_async(await browser.new_page())

        await page.goto(url)

        SEARCH_QUERY = """
        {
            news_articles
        }
        """

        NEWSROOM_OVERVIEW = """
        {
            news_articles[] {
                title
                body
                date
                link
            }
        }
        """
        # Get the response
        response = await page.query_data(NEWSROOM_OVERVIEW)

        # Print the response to inspect its structure
        print(response)

        # Access the articles from the response, if available
        articles = response.get('news_articles', [])

        await browser.close()

        # Return the company's name and the extracted news articles
        return {
            "company": company,
            "news_articles": articles
        }

# Process Companies Data

In [ ]:
# Main function to process the Excel and scrape data
async def process_companies_data(excel_file_path):
    companies_data = load_companies_data(excel_file_path)

    # List to hold the results for all companies
    all_companies_news = []

    # Scrape news articles for each company
    for _, row in companies_data.iterrows():
        company_name = row['company']
        company_url = row['link']
        print(f"Scraping news articles for: {company_name} ({company_url})")

        company_news = await scrape_news_articles_for_company(company_name, company_url)
        all_companies_news.append(company_news)

    # Convert the data to a DataFrame
    df = pd.json_normalize(all_companies_news, "news_articles", ["company"])

    # Convert the DataFrame to JSON for structured output
    structured_json = df.to_json(orient="records", lines=True)

    return df, structured_json

# Execute Code

In [24]:
import asyncio

loop = asyncio.get_running_loop()

excel_file_path = "/content/competitor_newsroom.xlsx"
task = loop.create_task(process_companies_data(excel_file_path))
df, structured_json = await task

# Display DataFrame and JSON result
print(structured_json)

Scraping news articles for: Accenture (https://newsroom.accenture.com/)
{'news_articles': [{'title': 'Accenture and Schaeffler Pave the Way for Industrial Humanoid Robots with NVIDIA and Microsoft Technologies', 'body': 'HANOVER; Germany; April 1, 2025 – Accenture (NYSE: ACN) is working with Schaeffler AG to reinvent industrial automation with physical AI and robotics.', 'date': 'April 01, 2025', 'link': 'https://newsroom.accenture.com/news/2025/accenture-and-schaeffler-pave-the-way-for-industrial-humanoid-robots-with-nvidia-and-microsoft-technologies'}, {'title': 'Malabar Gold & Diamonds Collaborates with Accenture to Accelerate AI-Driven Business Reinvention', 'body': 'NEW DELHI; April 1, 2025 – Malabar Gold & Diamonds, the sixth largest jewelry retailer globally with over 380 showrooms across 13 countries, is collaborating with Accenture (NYSE: ACN) to accelerate its technology reinvention through an AI-powered, cloud enabled digital core. This initiative aims to elevate the custome

# DynamoDB Access

In [26]:
import boto3
import pandas as pd
from google.colab import userdata

# AWS-Zugangsdaten (ersetze sie durch deine)
AWS_ACCESS_KEY = userdata.get('AWS_ACCESS_KEY')
AWS_SECRET_KEY = userdata.get('AWS_SECRET_KEY')
REGION_NAME = userdata.get('REGION_NAME')

# Verbindung zur DynamoDB herstellen
dynamodb = boto3.resource(
    "dynamodb",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=REGION_NAME,
)

In [ ]:
# Daten in DynamoDB hochladen
for _, row in df.iterrows():
    item = {
        "title": row["title"],
        "body": row["body"],
        "date": row["date"],
        "link": row["link"],  # Muss der PRIMARY KEY der DynamoDB-Tabelle sein
        "company": row["company"],
    }

    table.put_item(Item=item)

    print("Data successfully uploaded to DynamoDB!")

# Get existing data

In [27]:
import boto3
import pandas as pd
from datetime import datetime
from boto3.dynamodb.conditions import Key, Attr

# Connect to DynamoDB
dynamodb = boto3.resource(
    "dynamodb",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=REGION_NAME,
)

# Assuming your table name is 'company_news'
table = dynamodb.Table('NewsArticles')

# Your scraped news data (already in a DataFrame called df_)
scraped_news_df = df  # This is the DataFrame you showed

def get_existing_news_for_company(company_name):
    """Retrieve all existing news for a specific company from DynamoDB using scan with filter"""
    response = table.scan(
        FilterExpression=Attr('company').eq(company_name)
    )

    items = response['Items']

    # Handle pagination if there are more items
    while 'LastEvaluatedKey' in response:
        response = table.scan(
            FilterExpression=Attr('company').eq(company_name),
            ExclusiveStartKey=response['LastEvaluatedKey']
        )
        items.extend(response['Items'])

    if items:
        return pd.DataFrame(items)
    else:
        return pd.DataFrame(columns=['title', 'body', 'date', 'link', 'company'])

In [28]:
df_exisiting_news = get_existing_news_for_company('Accenture')